### 1.0 Libraries and directories

In [1]:
import glob
import pprint as pp
import pandas as pd
import geopandas as gpd
import ee
import geemap

ee.Authenticate()
ee.Initialize()

In [2]:
rois = ee.data.listAssets('projects/alpod-412314/assets/ROIs/')
rois_ids = [roi['id'] for roi in rois['assets']]
rois_ids.remove('projects/alpod-412314/assets/ROIs/all_regions')
pp.pp(rois_ids)


gswo_monthly_recurrence = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
june = 6
aug = 8
june_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', june)).first()
aug_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', june)).first()

['projects/alpod-412314/assets/ROIs/AKCP_roi',
 'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi',
 'projects/alpod-412314/assets/ROIs/MRD_roi',
 'projects/alpod-412314/assets/ROIs/TUK_roi',
 'projects/alpod-412314/assets/ROIs/YKdelta_roi',
 'projects/alpod-412314/assets/ROIs/YKflats_roi',
 'projects/alpod-412314/assets/ROIs/anderson_plain_roi']


In [3]:
def export_rois(roi_id, target_img, month_name):
    
    roi_name_temp = roi_id.split('/')[-1]
    roi_name = roi_name_temp.split('_roi')[0]
    feature = ee.FeatureCollection(roi_id).first()
    bounds = feature.geometry()

    clipped_img = target_img.clip(bounds)

    export_params = {
        'image': clipped_img,
        'description': f'GSWORecurrence_{roi_name}_month_{month_name}_', 
        'fileNamePrefix': f'GSWORecurrence_{roi_name}_month_{month_name}_',
        'folder': 'gswo_exports', 
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': bounds,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    }

    ee.batch.Export.image.toDrive(**export_params).start()

In [4]:
datasets = [june_data, aug_data]
month_names = ['june', 'aug']

monthly_datasets = list(zip(month_names, datasets))

for month, ds in monthly_datasets:
    for roi_id in rois_ids:
        export_rois(roi_id, ds, month)